In [ ]:
# This notebook is used to analyse the best combinis in detail. All the tests with length, noise etc. are being analysed here.
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import iqr

from src.main.rl.evaluation.plots.phase3_plots import (
    plot_actions_taken,
    plot_observations,
)
from src.main.rl.evaluation.phase3_evaluation import create_evaluation_df_phase3
from src.main.rl.utils.constants import color_mapping

paths = [
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06",
    "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario2/training_04_06/scenario2_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_None_RewardOption2Wrapper_PPO_training_04_06",
    "src/main/rl/models/scenario3/training_04_06/scenario3_ActionSpaceOption3Wrapper_ObservationOption3Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_PPO_training_04_06",
]
path_to_save = "src/main/rl/evaluation/output/phase3_evaluation_results.csv"
pd.options.display.max_colwidth = 500
df = pd.DataFrame()
try:
    df = pd.read_csv(path_to_save)
except:
    pass
if df.empty:
    create_evaluation_df_phase3(path_to_save, paths)
    df = pd.read_csv(path_to_save)
len(df)

In [ ]:
# Statistics per combination
df_statistics_per_combination = (
    df.drop(columns=["full_path"])
    .groupby(
        [
            "combination",
            "scenario",
            "alg",
            "action_wrapper",
            "obs_wrapper",
            "automation_wrapper",
        ],
        dropna=False,
    )
    .agg(["mean", "max", "min", "std", iqr])
)
# Necessary to set index as alphabetical is confusing for the thesis
df_statistics_per_combination["index"] = [0, 1, 3, 2, 5, 4]
df_statistics_per_combination = (
    df_statistics_per_combination.reset_index()
    .set_index(["index", "combination"])
    .sort_index()
)
df_statistics_per_combination.columns = [
    "_".join(a) for a in df_statistics_per_combination.columns.to_flat_index()
]
save_df_per_combination = df_statistics_per_combination.copy()
df_statistics_per_combination

In [ ]:
# SOP Analysis
from src.main.rl.evaluation.eval import evaluate_sop

(
    cum_reward_sop,
    criticality_score_sop,
    total_timesteps_sop,
    actions_taken_sop,
    obs_taken_sop,
    info_sop,
) = evaluate_sop()

In [ ]:
y_axis_scale_actions = [[0, 100], [0, 2200], [0, 1.1], [0, 1.1], [0, 2200]]
fig = plot_actions_taken(actions_taken_sop, "scenario1", y_axis_scale_actions)
# fig.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_action_sop.png",
#    format="png",
#    dpi=300,
# )

In [ ]:
y_axis_scale_obs = [[0, 1000], [0, 4000], [0, 550], [0, 8000], [0, 180], [0, 32]]
fig = plot_observations(obs_taken_sop, y_axis_scale_obs)
# fig.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_obs_sop.png",
#    format="png",
#    dpi=300,
# )

In [ ]:
# 1000 length
cols = [item for item in df_statistics_per_combination.columns if "1000" in item]
df_statistics_per_combination[cols].round(2)

In [ ]:
# Criticality Score per combination
print(df_statistics_per_combination[["episode_length_1000_mean"]].round(2))


print(
    df_statistics_per_combination[cols]
    .round(2)
    .droplevel(1)[
        [
            "episode_length_1000_mean",
            "episode_length_1000_max",
            "episode_length_1000_std",
            "episode_length_1000_iqr",
            "episode_length_1000_timesteps_mean",
        ]
    ]
    .transpose()
    .to_latex()
)

In [ ]:
# Check how often the pumps are blown in the 1000 length test
df[
    [
        "episode_length_1000_condensator_pump_blown",
        "episode_length_1000_water_pump_blown",
    ]
].sum()

In [ ]:
# Criticality Score in more detail
cols = [item for item in df_statistics_per_combination.columns if "criti" in item]
df_statistics_per_combination[cols].round(2)

In [ ]:
# Noise
cols = [item for item in df_statistics_per_combination.columns if "DelayNoise" in item]
df_statistics_per_combination[cols].round(2)

print(df_statistics_per_combination[["DelayNoiseWrapperOption1_mean"]].round(2))
print(
    df_statistics_per_combination[cols]
    .round(2)
    .droplevel(1)[
        [
            "DelayNoiseWrapperOption1_mean",
            "DelayNoiseWrapperOption1_max",
            "DelayNoiseWrapperOption1_min",
            "DelayNoiseWrapperOption1_std",
            "DelayNoiseWrapperOption1_iqr",
            "DelayNoiseWrapperOption2_mean",
            "DelayNoiseWrapperOption2_max",
            "DelayNoiseWrapperOption2_min",
            "DelayNoiseWrapperOption2_std",
            "DelayNoiseWrapperOption2_iqr",
        ]
    ]
    .transpose()
    .to_latex()
)

In [ ]:
# Big noise box plot
delay_varies_noise = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "DelayNoiseWrapperOption2",
        "DelayNoiseWrapperOption1",
        "combination",
        "ObservationVariesNoiseWrapper1",
        "ObservationVariesPositiveNoiseWrapper",
        "ObservationVariesNegativeNoiseWrapper",
    ]
]
fig, ax = plt.subplots()
fig.set_figheight(3)
fig.set_figwidth(8)
for idx, item in enumerate(delay_varies_noise["combination"].unique()):
    ax.boxplot(
        delay_varies_noise.query(f"combination=='{item}'")[
            ["DelayNoiseWrapperOption1", "DelayNoiseWrapperOption2"]
        ],
        positions=[idx, idx + 0.2],
    )


plt.show()

In [ ]:
# Box Plots for nose per Option and per Scenario -> Return
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


order_of_col = [
    "ObservationVariesPositiveNoiseWrapper",
    "ObservationVariesNegativeNoiseWrapper",
    "ObservationVariesNoiseWrapper1",
    "DelayNoiseWrapperOption1",
    "DelayNoiseWrapperOption2",
]
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario1:Scenario1'")[
        order_of_col
    ],
    positions=[0, 0.2, 0.4, 0.6, 0.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax2 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario2:Scenario2'")[
        order_of_col
    ],
    positions=[1.2, 1.4, 1.6, 1.8, 2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax3 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario3:Scenario3'")[
        order_of_col
    ],
    positions=[2.4, 2.6, 2.8, 3, 3.2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
set_box_color(ax1, color_mapping["scenario1"], 0)
set_box_color(ax1, color_mapping["scenario1"], 1)
set_box_color(ax2, color_mapping["scenario2"], 0)
set_box_color(ax2, color_mapping["scenario2"], 1)
set_box_color(ax3, color_mapping["scenario3"], 0)
set_box_color(ax3, color_mapping["scenario3"], 1)
ax.set_xlabel("Noise Options")
ax.set_ylabel("Return")
plt.plot([], c=color_mapping["scenario1"], label="Scenario 1")
plt.plot([], c=color_mapping["scenario2"], label="Scenario 2")
plt.plot([], c=color_mapping["scenario3"], label="Scenario 3")
plt.legend()
plt.title("Rauschen im Bezug auf die Szenarien")
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_noise_by_scen.png",
    format="png",
    dpi=300,
)

In [ ]:
# Box Plots for noise per option and per scenario -> timesteps
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


delay_varies_noise = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "DelayNoiseWrapperOption2_timesteps",
        "DelayNoiseWrapperOption1_timesteps",
        "combination",
        "ObservationVariesNoiseWrapper1_timesteps",
        "ObservationVariesPositiveNoiseWrapper_timesteps",
        "ObservationVariesNegativeNoiseWrapper_timesteps",
    ]
]

order_of_col = [
    "ObservationVariesPositiveNoiseWrapper_timesteps",
    "ObservationVariesNegativeNoiseWrapper_timesteps",
    "ObservationVariesNoiseWrapper1_timesteps",
    "DelayNoiseWrapperOption1_timesteps",
    "DelayNoiseWrapperOption2_timesteps",
]
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario1:Scenario1'")[
        order_of_col
    ],
    positions=[0, 0.2, 0.4, 0.6, 0.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax2 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario2:Scenario2'")[
        order_of_col
    ],
    positions=[1.2, 1.4, 1.6, 1.8, 2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax3 = ax.boxplot(
    delay_varies_noise.query("scenario=='src.main.rl.envs.scenario3:Scenario3'")[
        order_of_col
    ],
    positions=[2.4, 2.6, 2.8, 3, 3.2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
set_box_color(ax1, color_mapping["scenario1"], 0)
set_box_color(ax1, color_mapping["scenario1"], 1)
set_box_color(ax2, color_mapping["scenario2"], 0)
set_box_color(ax2, color_mapping["scenario2"], 1)
set_box_color(ax3, color_mapping["scenario3"], 0)
set_box_color(ax3, color_mapping["scenario3"], 1)
ax.set_xlabel("Noise Options")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["scenario1"], label="Scenario 1")
plt.plot([], c=color_mapping["scenario2"], label="Scenario 2")
plt.plot([], c=color_mapping["scenario3"], label="Scenario 3")
plt.legend()
plt.title("Rauschen im Bezug die Szenarien")
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_noise_by_scen.png",
    format="png",
    dpi=300,
)

In [ ]:
# Box Plots for noise per option and per automation -> timesteps
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    delay_varies_noise.query("automation_wrapper.isna()")[order_of_col],
    positions=[0, 0.2, 0.4, 0.6, 0.8],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
ax2 = ax.boxplot(
    delay_varies_noise.query("automation_wrapper=='NPPAutomationWrapper'")[
        order_of_col
    ],
    positions=[1.2, 1.4, 1.6, 1.8, 2],
    labels=["1a", "1b", "1c", "2a", "2b"],
)
set_box_color(ax1, color_mapping["red"], 0)
set_box_color(ax1, color_mapping["red"], 1)
set_box_color(ax2, color_mapping["NPPAutomationWrapper"], 0)
set_box_color(ax2, color_mapping["NPPAutomationWrapper"], 1)

ax.set_xlabel("Noise Options")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["NPPAutomationWrapper"], label="NPPAutomation aktiviert")
plt.plot([], c=color_mapping["red"], label="NPPAutomation deaktiviert")
plt.legend()
plt.title("Rauschen im Bezug auf NPPAutomation")
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_noise_by_automation.png",
    format="png",
    dpi=300,
)

In [ ]:
# Check combination 1 model 3 for noise and why it fails in detail (plotting)
import pandas as pd
from src.main.rl.utils.parser import parse_wrapper
from src.main.rl.utils.combined_parser import parse_information_from_path
from src.main.rl.evaluation.eval import evaluate
from src.main.rl.utils.utils import WrapperMaker
from src.main.rl.evaluation.wrapper.noise.obs_varies_wrapper import (
    ObservationVariesNegativeNoiseWrapper,
    ObservationVariesPositiveNoiseWrapper,
)
from src.main.rl.evaluation.plots.phase3_plots import (
    plot_observations,
    plot_actions_taken,
)
from src.main.rl.evaluation.eval import evaluate, evaluate_terminal_state_obs

path = "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06_3"

path_to_overhand = path + "/best_model.zip"

action_wrapper, automation_wrapper, obs_wrapper, reward_wrapper = parse_wrapper(path)
scenario, alg, wrapper_maker = parse_information_from_path(path)
wrapper_maker = WrapperMaker(
    action_wrapper,
    automation_wrapper,
    obs_wrapper,
    reward_wrapper,
    None,
    ObservationVariesNegativeNoiseWrapper,
)
(
    cum_reward,
    criticality_score,
    total_timesteps,
    actions_taken,
    obs_taken,
    info,
) = evaluate(scenario, path_to_overhand, alg, wrapper_maker, episode_length=5)
cum_reward2, obs_taken2, info2 = evaluate_terminal_state_obs(
    scenario, path_to_overhand, alg, wrapper_maker, episode_length=5
)

assert cum_reward2 == cum_reward
y_axis_scale_obs = [[0, 1000], [0, 4000], [0, 550], [0, 8000], [0, 180], [0, 32]]

fig = plot_observations(obs_taken2, y_axis_scale_obs)
fig1 = plot_observations(obs_taken, y_axis_scale_obs)
fig2 = plot_actions_taken(actions_taken, "scenario1")
# fig2.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_noise_combi_1_action_not_working.png",
#    format="png",
#    dpi=300,
# )

In [ ]:
wrapper_maker = WrapperMaker(
    action_wrapper, automation_wrapper, obs_wrapper, reward_wrapper, None, None
)
(
    cum_reward,
    criticality_score,
    total_timesteps,
    actions_taken,
    obs_taken,
    info,
) = evaluate(scenario, path_to_overhand, alg, wrapper_maker, episode_length=2)
y_axis_scale_obs = [[0, 1000], [0, 4000], [0, 550], [0, 8000], [0, 180], [0, 32]]
# fig=plot_observations(obs_taken,y_axis_scale_obs)
# fig2=plot_actions_taken(actions_taken, "scenario1")
# fig2.savefig(
#    f"src/main/rl/evaluation/plot_results/phase3_noise_combi_1_action_not_working_comp_with_working.png",
#    format="png",
#    dpi=300,
# )

In [ ]:
# Starting State
cols = [col for col in df.columns if "starting" in col and "criticality" not in col]
df[cols].mean()

In [ ]:
# Starting State Option 1
starting_state1 = df[
    [
        "create_starting_state_option1",
        "create_starting_state_option1_timesteps",
        "automation_wrapper",
    ]
]
starting_state1.loc[
    starting_state1["automation_wrapper"].isna(), "automation_wrapper"
] = "NaN"
starting_state1.groupby("automation_wrapper").agg("mean")

In [ ]:
# Check combination 1 model 8 for noise and why it fails in detail (plotting)
import pandas as pd
from src.main.rl.utils.parser import parse_wrapper
from src.main.rl.utils.combined_parser import parse_information_from_path
from src.main.rl.evaluation.eval import evaluate
from src.main.rl.utils.utils import WrapperMaker
from src.main.rl.evaluation.wrapper.noise.obs_varies_wrapper import (
    ObservationVariesNegativeNoiseWrapper,
    ObservationVariesPositiveNoiseWrapper,
)
from src.main.rl.evaluation.plots.phase3_plots import (
    plot_observations,
    plot_actions_taken,
)
from src.main.rl.evaluation.eval import evaluate, evaluate_terminal_state_obs
from src.main.rl.utils.constants import (
    ALL_OBS_NOISE_WRAPPERS,
    ALL_DELAY_NOISE_WRAPPERS,
    STARTING_STATE_OPTION1,
    STARTING_STATE_OPTION2,
    STARTING_STATE_OPTION3,
)

path = "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption4Wrapper_None_RewardOption2Wrapper_TD3_training_04_06_8"
# path = "src/main/rl/models/scenario1/training_04_06/scenario1_ActionSpaceOption3Wrapper_ObservationOption5Wrapper_NPPAutomationWrapper_RewardOption2Wrapper_SAC_training_04_06_5"

path_to_overhand = path + "/best_model.zip"

action_wrapper, automation_wrapper, obs_wrapper, reward_wrapper = parse_wrapper(path)
scenario, alg, wrapper_maker = parse_information_from_path(path)
wrapper_maker = WrapperMaker(
    action_wrapper, automation_wrapper, obs_wrapper, reward_wrapper, None, None
)
(
    cum_reward,
    criticality_score,
    total_timesteps,
    actions_taken,
    obs_taken,
    info,
) = evaluate(
    scenario,
    path_to_overhand,
    alg,
    wrapper_maker,
    episode_length=250,
    starting_state=STARTING_STATE_OPTION1[0](),
)
cum_reward2, obs_taken2, info2 = evaluate_terminal_state_obs(
    scenario,
    path_to_overhand,
    alg,
    wrapper_maker,
    episode_length=250,
    starting_state=STARTING_STATE_OPTION1[0](),
)

assert cum_reward2 == cum_reward
y_axis_scale_obs = [[0, 1000], [0, 4000], [0, 550], [0, 8000], [0, 180], [0, 32]]

# TODO if needed plot also the changed obs into the graphs
fig = plot_observations(obs_taken, y_axis_scale_obs)
print(fig)
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state1_combi2_obs.png",
    format="png",
    dpi=300,
)
fig2 = plot_actions_taken(actions_taken, "scenario1")

fig2.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state1_combi2_act.png",
    format="png",
    dpi=300,
)

In [ ]:
df[
    [
        "create_starting_state_option2a_timesteps",
        "create_starting_state_option2b_timesteps",
        "create_starting_state_option2c_timesteps",
    ]
]

In [ ]:
# Box Plots for starting state per option and per scenario -> timesteps
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


starting_state2 = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "create_starting_state_option2a_timesteps",
        "create_starting_state_option2b_timesteps",
        "create_starting_state_option2c_timesteps",
    ]
]

order_of_col = [
    "create_starting_state_option2a_timesteps",
    "create_starting_state_option2b_timesteps",
    "create_starting_state_option2c_timesteps",
]
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    starting_state2.query("scenario=='src.main.rl.envs.scenario1:Scenario1'")[
        order_of_col
    ],
    positions=[
        0,
        0.2,
        0.4,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax2 = ax.boxplot(
    starting_state2.query("scenario=='src.main.rl.envs.scenario2:Scenario2'")[
        order_of_col
    ],
    positions=[
        0.9,
        1.1,
        1.3,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax3 = ax.boxplot(
    starting_state2.query("scenario=='src.main.rl.envs.scenario3:Scenario3'")[
        order_of_col
    ],
    positions=[1.8, 2, 2.2],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
set_box_color(ax1, color_mapping["scenario1"], 0)
set_box_color(ax1, color_mapping["scenario1"], 1)
set_box_color(ax2, color_mapping["scenario2"], 0)
set_box_color(ax2, color_mapping["scenario2"], 1)
set_box_color(ax3, color_mapping["scenario3"], 0)
set_box_color(ax3, color_mapping["scenario3"], 1)
ax.set_xlabel("Startzustände")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["scenario1"], label="Scenario 1")
plt.plot([], c=color_mapping["scenario2"], label="Scenario 2")
plt.plot([], c=color_mapping["scenario3"], label="Scenario 3")
plt.legend()
plt.title("Einfluss der Startzustände der Option 2 (Szenarien)")
plt.show()
# TODO if necessary change scenario and Startzustände
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state_2_by_scen.png",
    format="png",
    dpi=300,
)

In [ ]:
starting_state2.groupby("scenario").agg("mean")

In [ ]:
# Box Plots for starting state per option and per automation -> timesteps
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    starting_state2.query("automation_wrapper.isna()")[order_of_col],
    positions=[
        0,
        0.2,
        0.4,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax2 = ax.boxplot(
    starting_state2.query("automation_wrapper=='NPPAutomationWrapper'")[order_of_col],
    positions=[
        0.9,
        1.1,
        1.3,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
set_box_color(ax1, color_mapping["red"], 0)
set_box_color(ax1, color_mapping["red"], 1)
set_box_color(ax2, color_mapping["NPPAutomationWrapper"], 1)

ax.set_xlabel("Startzustände")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["NPPAutomationWrapper"], label="NPPAutomation aktiviert")
plt.plot([], c=color_mapping["red"], label="NPPAutomation deaktiviert")
plt.legend()
plt.title("Einfluss der Startzustände der Option 2 (NPPAutomation)")
plt.show()
fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state_2_by_automation.png",
    format="png",
    dpi=300,
)

In [ ]:
# Starting state 2 timesteps with/withou automation
starting_state2.loc[
    starting_state2["automation_wrapper"].isna(), "automation_wrapper"
] = "NaN"
starting_state2.groupby("automation_wrapper").agg("mean")

In [ ]:
# Box Plots for starting state per option and per scenario -> timesteps
def set_box_color(bp, color, idx):
    plt.setp(bp["boxes"], color=color)
    plt.setp(bp["whiskers"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["medians"], color=color)
    plt.setp(bp["caps"], color=color)
    plt.setp(bp["fliers"], color=color)


starting_state2 = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "create_starting_state_option2a_timesteps",
        "create_starting_state_option2b_timesteps",
        "create_starting_state_option2c_timesteps",
    ]
]

order_of_col = [
    "create_starting_state_option2a_timesteps",
    "create_starting_state_option2b_timesteps",
    "create_starting_state_option2c_timesteps",
]
fig, ax = plt.subplots()
ax1 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario1:Scenario1' and automation_wrapper.isna()"
    )[order_of_col],
    positions=[
        0,
        0.2,
        0.4,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax2 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario1:Scenario1' and automation_wrapper=='NPPAutomationWrapper'"
    )[order_of_col],
    positions=[
        0.8,
        1,
        1.2,
    ],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax3 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario2:Scenario2' and automation_wrapper.isna()"
    )[order_of_col],
    positions=[2, 2.2, 2.4],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax4 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario2:Scenario2' and automation_wrapper=='NPPAutomationWrapper'"
    )[order_of_col],
    positions=[2.8, 3, 3.2],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax5 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario3:Scenario3' and automation_wrapper.isna()"
    )[order_of_col],
    positions=[4, 4.2, 4.4],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
ax6 = ax.boxplot(
    starting_state2.query(
        "scenario=='src.main.rl.envs.scenario3:Scenario3' and automation_wrapper=='NPPAutomationWrapper'"
    )[order_of_col],
    positions=[4.8, 5, 5.2],
    labels=[
        "2a",
        "2b",
        "2c",
    ],
)
set_box_color(ax1, color_mapping["scenario1"], 0)
set_box_color(ax1, color_mapping["scenario1"], 1)
set_box_color(ax2, color_mapping["scenario2"], 0)
set_box_color(ax2, color_mapping["scenario2"], 1)
set_box_color(ax3, color_mapping["scenario3"], 0)
set_box_color(ax3, color_mapping["scenario3"], 1)
ax.set_xlabel("Startzustände")
ax.set_ylabel("Absolvierte Zeitschritte")
plt.plot([], c=color_mapping["scenario1"], label="Scenario 1")
plt.plot([], c=color_mapping["scenario2"], label="Scenario 2")
plt.plot([], c=color_mapping["scenario3"], label="Scenario 3")
plt.legend()
plt.title("Einfluss der Startzustände der Option 2 (Szenarien)")
plt.show()

fig.savefig(
    f"src/main/rl/evaluation/plot_results/phase3_starting_state_2_by_scen.png",
    format="png",
    dpi=300,
)

In [ ]:
starting_state3 = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "create_starting_state_option3_timesteps",
    ]
]
starting_state3.loc[
    starting_state3["automation_wrapper"].isna(), "automation_wrapper"
] = "NaN"
starting_state3.groupby("automation_wrapper").agg("mean")

In [ ]:
starting_state3.groupby("scenario").agg("mean")

In [ ]:
df_statistics_per_combination[
    [
        "create_starting_state_option1_timesteps_mean",
        "create_starting_state_option2a_timesteps_mean",
        "create_starting_state_option2b_timesteps_mean",
        "create_starting_state_option2c_timesteps_mean",
        "create_starting_state_option3_timesteps_mean",
    ]
].agg(["mean", "max", "min", "std"], axis=1)

In [ ]:
print(
    df_statistics_per_combination[
        [
            "create_starting_state_option1_timesteps_mean",
            "create_starting_state_option2a_timesteps_mean",
            "create_starting_state_option2b_timesteps_mean",
            "create_starting_state_option2c_timesteps_mean",
            "create_starting_state_option3_timesteps_mean",
        ]
    ]
    .agg(["mean", "max", "min", "std"], axis=1)
    .droplevel(1)
    .round(2)
    .transpose()
    .to_latex()
)

In [ ]:
# TODO t-test? bzw ANOVA

In [ ]:
starting_state_total = df[
    [
        "obs_wrapper",
        "automation_wrapper",
        "scenario",
        "create_starting_state_option1_timesteps",
        "create_starting_state_option2a_timesteps",
        "create_starting_state_option2b_timesteps",
        "create_starting_state_option2c_timesteps",
        "create_starting_state_option3_timesteps",
    ]
]

In [ ]:
# Starting state comparison grouped by different categories e.g. scenario by  timesteps
starting_state_total.loc[
    starting_state_total["automation_wrapper"].isna(), "automation_wrapper"
] = "NaN"
starting_state_total.groupby("scenario").agg(["mean"], axis=0).round(2).agg(
    "mean", axis=1
)

In [ ]:
starting_state_total.loc[
    starting_state_total["automation_wrapper"].isna(), "automation_wrapper"
] = "NaN"
starting_state_total.groupby("automation_wrapper").agg(["mean"], axis=0).round(2).agg(
    "mean", axis=1
)

In [ ]:
# T-test (Welch)
from scipy.stats import ttest_ind

wo_NPP = starting_state_total[starting_state_total["automation_wrapper"] == "NaN"].drop(
    columns=["obs_wrapper", "scenario", "automation_wrapper"]
)
wo_NPP = wo_NPP.stack().reset_index(drop=True)
w_NPP = starting_state_total[
    starting_state_total["automation_wrapper"] == "NPPAutomationWrapper"
].drop(columns=["obs_wrapper", "scenario", "automation_wrapper"])
w_NPP = w_NPP.stack().reset_index(drop=True)

ttest_ind(wo_NPP, w_NPP, equal_var=False)

In [ ]:
wo_NPP = starting_state_total[starting_state_total["automation_wrapper"] == "NaN"].drop(
    columns=["obs_wrapper", "scenario", "automation_wrapper"]
)["create_starting_state_option3_timesteps"]
w_NPP = starting_state_total[
    starting_state_total["automation_wrapper"] == "NPPAutomationWrapper"
].drop(columns=["obs_wrapper", "scenario", "automation_wrapper"])[
    "create_starting_state_option3_timesteps"
]
ttest_ind(wo_NPP, w_NPP, equal_var=False)

In [ ]:
starting_state_total.eq(250).sum().sum()

In [ ]:
# NPP Automation activated/deactivated
[col for col in df.columns if "npp" in col]
df[
    [
        "result_wo_npp_automation",
        "result_w_npp_automation",
        "automation_wrapper",
        "scenario",
    ]
].max()